Que es un ORM?
Un ORM, por sus siglas al inglés: Object Relational Mappper, no es más que una pieza de software que nos permite interactuar con nuestra base de datos sin la necesidad de conocer SQL (El lenguaje de consultas). Todo esto utilizando el paradigma de programación orientada a objetos. 

Los ORMs se encarga de traducir nuestra instrucción en el lenguaje de programación que estemos utilizando a una sentencia SQL que el gestor de base de datos pueda entender.

Veamos un ejemplo.
Que pasa si deseamos obtener el id, username, email para nuestro usuarios con id número 1.

Utilizando una base de datos relacional, MySQL por ejemplo. Nuestra consulta quedaría de la siguiente manera:
SELECT id, username, email FROM id = 1

Utilizando un ORM, por ejemplo PeeWee, quedaría de la siguiente manera y obteniendo el mismo resultado:
User.select(User.id, User.username, User.email).where(User.id == 1)

PeeWee es un ORM “ligero” que nos permite interactuar con bases de datos SQLite, MySQL, PostgreSQL.
La documentación de PeeWee está bastante completa y nos indica con ejemplos cómo utilizar este módulo en detalle. 
PeeWee es como SQLAlchemy, un ORN más “ligero” y “sencillo” de utilizar; con él podemos crear un modelo, insertar datos, consultar datos y por supuesto actualizar y eliminar los datos previamente introducidos. 
Soporta el uso de JOIN, claves foráneas y claves primarias compuestas.

In [ ]:
pip install peewee

In [ ]:
%load_ext autoreload
%autoreload 2

Creación de modelos

Declaramos la primera tabla llamada “Profesores” heredado de “Model”. 
El nombre de la clase será el nombre de nuestra tabla dentro de la base de datos.

Con los siguientes campos:
maestro_id como clave primaria del tipo AutoField para auto-increment el campo.
nombre, apellido, telefono e email como CharField para almacenar los datos tipo VarChar.
En email vemos que colocamos la opción de unique=True para que dicho campo sea único en el modelo.

A continuación declararmos la tabla de “Clases”, que igualmente heredamos de modelo, y las diferencias con lo que hemos visto antes son las siguientes:

Campo de fecha_inicio_curso y fecha_fin_curso, son del tipo DateField para almacenar (por defecto) formato Año-Mes-Día.
maestro_id, el cual es llave foránea del maestro_id de la tabla “Profesores”.

In [ ]:
from peewee import MySQLDatabase, AutoField, CharField, DateField, ForeignKeyField, Model

db = MySQLDatabase('pil2023', user='root', password='Pil2023', host='localhost', port=3306)

class Profesores(Model):
   maestro_id = AutoField()
   nombre = CharField()
   apellido = CharField()
   telefono = CharField()
   email = CharField(unique=True)

   class Meta:
       database = db

class Clases(Model):
   clase_id = AutoField()
   cod_curso = CharField()
   fecha_inicio_curso = DateField()
   fecha_fin_curso = DateField()
   horario = CharField()
   maestro_id = ForeignKeyField(Profesores)

   class Meta:
       database = db

db.connect()
db.create_tables([Profesores, Clases])

Insertar registros simples:


In [ ]:
nuevo_profe = Profesores( nombre='Luis',
                   apellido='Alanis',
                   telefono='3512542577',
                   email='luis.alanis@mail.com')
nuevo_profe.save()

In [ ]:
curso_pil2023 = Clases( cod_curso='python_db_2023', fecha_inicio_curso='2023-08-01', fecha_fin_curso='2023-12-01', horario='Noche', maestro_id = 1)
curso_pil2023.save()

Insertar registros múltiples

In [ ]:
cursos = [
   {'cod_curso': 'python_fullstack_2023', 'fecha_inicio_curso': '2023-08-01', 'fecha_fin_curso': '2023-12-01', 'horario': 'Noche', 'maestro_id': 1},
   {'cod_curso': 'python_net_2023', 'fecha_inicio_curso': '2023-08-01', 'fecha_fin_curso': '2023-12-01', 'horario': 'Noche', 'maestro_id': 1},
]

with db.atomic():
   query = Clases.insert_many(cursos)
   query.execute()

Recuperar registros

In [ ]:
for profesor in Profesores.select():
     print('Nombre: {} - Apellido: {} - Teléfono: {} - Email: {}'
     .format(profesor.nombre, profesor.apellido, profesor.telefono, profesor.email))

In [ ]:
profes = Profesores.select().where(Profesores.maestro_id==1)

for profesor in profes:
     print('Nombre: {} - Apellido: {} - Teléfono: {} - Email: {}'
     .format(profesor.nombre, profesor.apellido, profesor.telefono, profesor.email))


Recuperar registros relacionado las tablas

La sentencia en sql sería la sieguiente:
SELECT "t1"."maestro_id", "t1"."nombre", "t1"."apellido", "t1"."telefono", "t1"."email", "t2"."clase_id", "t2"."cod_curso", "t2"."fecha_inicio_curso", "t2"."fecha_fin_curso", "t2"."horario", "t2"."maestro_id" 
FROM "profesores" AS "t1" 
INNER JOIN "clases" AS "t2" ON ("t2"."maestro_id" = "t1"."maestro_id") 
WHERE ("t1"."maestro_id" = "t2"."maestro_id") GROUP BY "t2"."cod_curso"


In [ ]:
query = (Profesores
        .select(Profesores, Clases)
        .join(Clases)
        .group_by(Clases.cod_curso)
        .where(Profesores.maestro_id == Clases.maestro_id))

for curso in query:
   print('El curso {} esta comienza el {} y termina el {}, y va ser impartido por {} {}'
   .format(curso.clases.cod_curso, curso.clases.fecha_inicio_curso, curso.clases.fecha_fin_curso, curso.nombre, curso.apellido))

Actualización de registros

In [ ]:
actualizar_profe = Profesores.update(nombre='Alvaro').where(Profesores.maestro_id == 1)
actualizar_profe.execute()

Borrar datos

In [ ]:
eliminar_curso = Clases.delete().where(Clases.cod_curso == 'python_net_2023')
eliminar_curso.execute()